# PixelSounds: Visual Audio Encoding


## Introducción: Codificación de Audio en Vídeo con PixelSounds


PixelSounds es un sistema que convierte una señal de audio en un vídeo compuesto por frames. Cada frame codifica un bloque temporal del audio en forma de imagen, permitiendo visualizar la información sonora a través de representaciones visuales. El proceso es completamente reversible: se puede reconstruir el audio original a partir del vídeo.

Este enfoque permite investigar nuevas formas de percepción auditiva basadas en representación visual, y propone un esquema de codificación que, hasta donde sabemos, no está presente en herramientas o artículos convencionales de tratamiento de audio.

---

### Modos de codificación (`map_mode`)

Cada bloque de audio se transforma en una fila de píxeles mediante uno de los siguientes modos:

- **`ampl`**  
  Codifica la amplitud normalizada del audio en una escala de grises.  
  *Este modo ofrece la solución más simple y directa, útil como base de referencia.*

- **`fft`**  
  Codifica el espectro en frecuencia (magnitud y fase) usando la Transformada Rápida de Fourier (FFT). Requiere codificar dos componentes por muestra.  
  *El modo más completo y desafiante de reconstruir, ya que exige preservar fase.*

- **`fir`**  
  Separa el audio en tres bandas de frecuencia (baja, media y alta) usando filtros FIR, y codifica cada banda en uno de los canales de color (R, G, B).  
  *Una solución creativa para dividir el contenido espectral de manera intuitiva.*

---

### Modos de visualización (`color_mode`)

Los frames generados pueden representarse en dos formas:

- **`gris`**  
  Las filas codificadas se replican verticalmente como una imagen monocroma. Este modo se usa, por ejemplo, para visualizar la amplitud o una sola banda.  
  *Una buena opción para mantener fidelidad al contenido numérico.*

- **`color`**  
  Se colorea cada muestra según su valor en los tres canales RGB, permitiendo ver diferentes componentes (por ejemplo, magnitud y fase, o bandas FIR) en diferentes colores.  
  *Este modo explora la intersección entre percepción visual y estructuras del sonido.*

---

### Estructura del sistema

- **Codificador:**  
  Hace un análisis por bloques solapados y enventanados del audio, aplica la codificación elegida y genera una secuencia de imágenes PNG. Luego empaqueta los frames en un vídeo MP4 usando FFmpeg.  

- **Decodificador:**  
  Extrae los frames del vídeo, reconstruye los bloques de audio a partir de las imágenes, y aplica overlap-add para recuperar la señal completa.  
  *Permite probar si la codificación elegida conserva adecuadamente la señal.*

---

Esta estructura modular permite experimentar con distintas formas de representar el contenido sonoro, y abre la puerta a posibles usos en visualización interactiva, arte sonoro o codificación alternativa. Aunque centrado en una tarea técnica, el sistema introduce elementos creativos que conectan con otras disciplinas.


---


### Detalles del flujo del codificador

El proceso de codificación se inicia dividiendo el audio en bloques de tamaño fijo `N = fs / fps`, aplicando una ventana para suavizar los bordes, y generando una imagen por cada bloque.

Antes de comenzar con los datos, el sistema crea un **primer frame especial que contiene los metadatos** necesarios para la decodificación. Este frame de encabezado ocupa la posición `frame_0000.png` y codifica, en sus primeras filas, información crítica como:

- Tamaño del bloque `N` (dos bytes)
- Tamaño del salto `hop` entre bloques
- Frecuencia de vídeo `fps`
- Modo de visualización (`0` para gris, `1` para color)

Gracias a este diseño, no es necesario acompañar el vídeo de archivos auxiliares o configuraciones externas: todo lo necesario para la reconstrucción está embebido en el propio vídeo.

Después, para cada bloque de audio:

1. Se calcula la fila base de píxeles aplicando el modo de codificación (`map_mode`).
2. Dependiendo del modo de visualización (`color_mode`), esa fila se convierte en una imagen monocroma o en una representación coloreada.
3. Cada imagen se guarda como un fotograma numerado secuencialmente (`frame_0001.png`, `frame_0002.png`, etc.).
4. Finalmente, todas las imágenes se empaquetan en un archivo `.mp4` con ffmpeg.

Este flujo permite una representación audiovisual compacta, autosuficiente y fácilmente reversible del audio original.

Para garantizar que no haya pérdida de información entre los frames PNG generados y el vídeo final `.mp4`, el empaquetado se realiza con los siguientes parámetros de codificación:

- **Calidad constante sin pérdidas visuales:** se usa `-crf 0` o valores muy bajos para mantener fidelidad.
- **Submuestreo desactivado (4:4:4):** se fuerza `pix_fmt=yuv444p` para evitar la compresión cromática que degradaría los valores de color en `color_mode`.
- **Compresión intra-frame:** se evita la predicción entre frames, lo que permite acceder a cualquier frame de forma independiente sin depender de otros.

Esto asegura que el vídeo generado conserva exactamente los valores de píxeles de cada imagen, lo que es fundamental para que la reconstrucción del audio sea precisa y consistente.


In [6]:
# === IMPORTS ===
import os
import shutil
import subprocess
import numpy as np
import cv2
import imageio.v2 as imageio
from scipy.signal import firwin, lfilter, get_window
from scipy.fft import fft, ifft
from scipy.io import wavfile as wav
from IPython.display import Audio, Video, display

from vozyaudio import lee_audio, sonido

In [3]:
class PixelSoundsEncoder:
    """
    Convierte un audio en un “vídeo pixelado” codificando cada bloque de
    muestras en filas de imágenes.

    - Primer frame: metadatos (N, hop, fps, color_mode).
    - Siguientes frames: bloques de audio coloreados (gris o RGB).
    - Luego empaqueta PNGs en MP4 con un batch externo.
    """
    def __init__(
        self,
        audio_path,
        frames_dir="fotogramas",
        export_dir="exports",
        fps=60,
        color_mode="color",   # 'gris' o 'color'
        map_mode="ampl",       # 'ampl', 'fft' o 'fir'
        window_type="hann",
        numcoef=101
    ):
        self.audio_path  = audio_path
        self.frames_dir  = frames_dir
        self.export_dir  = export_dir
        self.fps         = fps
        self.color_mode  = color_mode
        self.map_mode    = map_mode
        self.window_type = window_type
        self.iscolor     = 1 if color_mode == "color" else 0

        # Leer y normalizar audio de 16-bit
        self.fs, audio = lee_audio(audio_path)
        self.audio     = audio.astype(np.float32)
        self.audio    /= np.max(np.abs(self.audio)) + 1e-12

        # Diseñar filtros FIR
        self.b_low  = firwin(numcoef,       cutoff=3000,                   fs=self.fs)
        self.b_band = firwin(numcoef, [3000,10000], pass_zero=False,         fs=self.fs)
        self.b_high = firwin(numcoef,       cutoff=10000, pass_zero=False,   fs=self.fs)

        # Crear carpetas de salida
        if os.path.exists(self.frames_dir):
            shutil.rmtree(self.frames_dir)
        os.makedirs(self.frames_dir, exist_ok=True)
        os.makedirs(self.export_dir, exist_ok=True)

    def _write_header(self, N, hop):
        """
        Escribe el frame 0000 como cabecera con metadatos de N, hop, fps y color_mode.
        """
        # crear imagen vacia NxN
        header = np.zeros((N, N), dtype=np.uint8)

        # codificar N y hop en bytes alto y bajo
        hiN, loN = (N>>8)&0xFF, N&0xFF
        hiH, loH = (hop>>8)&0xFF, hop&0xFF

        # rellenar filas con metadatos
        header[0,:] = hiN
        header[1,:] = loN
        header[2,:] = hiH
        header[3,:] = loH
        header[4,:] = self.fps & 0xFF         # fps
        header[5,:] = self.iscolor            # flag color

        # guardar como frame_0000
        path = os.path.join(self.frames_dir, "frame_0000.png")
        imageio.imwrite(path, header)

    def _colorear_fila(self, fila, modo):
        """
        Aplica mapeado de color a la fila segun el modo.

        - ampl → R = amp, G = 255-amp, B = constante
        - fft  → R = mag, G = phase, B = constante
        - fir  → R/G/B = bandas low/band/high

        Devuelve un array Nx3 uint8 (RGB por pixel).
        """

        if modo == 'ampl':
            # si es color, cogemos solo el canal rojo
            amp = fila[:,0] if fila.ndim == 2 else fila

            # rojo: valor original
            r = amp
            # verde: complementario para dar contraste
            g = 255 - amp
            # azul: constante (centro)
            b = 128 * np.ones_like(amp, dtype=np.uint8)

        elif modo == 'fft':
            # usamos magnitud (R) y fase (G), B fijo
            mag_n, phase_n = fila[:,0], fila[:,1]
            r = mag_n
            g = phase_n
            b = 255 * np.ones_like(r, dtype=np.uint8)  # canal azul a tope

        elif modo == 'fir':
            # ya vienen como low/band/high → R/G/B
            r, g, b = fila[:,0], fila[:,1], fila[:,2]

        else:
            raise ValueError(f"Modo desconocido para colorear: {modo}")

        # ensamblar los tres canales en un solo array (Nx3)
        return np.stack([r, g, b], axis=1).astype(np.uint8)

    def generate_frames(self):
        """
        Divide el audio en bloques, aplica codificacion y guarda cada uno como PNG.
        """
        # 1) Calcular tamaño de bloque y hop
        N   = self.fs // self.fps
        if N % 2: N += 1               # aseguramos par
        hop = N // 2

        # 2) Guardar cabecera
        self._write_header(N, hop)

        # 3) Ventana y num de bloques
        window   = get_window(self.window_type, N, fftbins=True)
        n_blocks = (len(self.audio) - N) // hop
        print(f"[Encoder] N={N}, HOP={hop}, FPS={self.fps}, Bloques={n_blocks}")

        # 4) Procesar bloque a bloque
        for i in range(n_blocks):
            start = i * hop
            block = self.audio[start:start+N] * window

            # 4.1) Codificar segun map_mode
            if self.map_mode == "ampl":
                # normalizar el bloque a [0,1] y escalar a 8 bits
                norm = (block - block.min()) / (block.max() - block.min() + 1e-12)
                fila = (norm * 255).astype(np.uint8)  # fila resultante, uint8

            elif self.map_mode == "fft":
                # calcular la FFT
                X       = fft(block, n=N)
                X       = np.fft.fftshift(X)          # centrar la FFT
                mag     = np.abs(X)                   # magnitud
                phase   = np.angle(X)                 # fase en radianes

                # normalizar magnitud a [0,255]
                mag_n   = np.round((mag / (mag.max() + 1e-12)) * 255).astype(np.uint8)
                # convertir fase de [-pi, pi] → [0, 255]
                phase_n = np.round(((phase + np.pi) / (2*np.pi)) * 255).astype(np.uint8)

                # construir fila RGB: R=mag, G=phase, B=0
                fila    = np.stack([mag_n, phase_n, np.zeros_like(mag_n)], axis=1)

            elif self.map_mode == "fir":
                # aplicar 3 filtros FIR: low, band y high
                y_l = lfilter(self.b_low,  1.0, block)
                y_b = lfilter(self.b_band, 1.0, block)
                y_h = lfilter(self.b_high, 1.0, block)

                # recortar cada señal a [-1,1]
                y_l, y_b, y_h = map(lambda y: np.clip(y, -1, 1), (y_l, y_b, y_h))

                # convertir a int8 → uint8 (para guardar como imagen)
                r8 = np.round(y_l * 127).astype(np.int8).view(np.uint8)
                g8 = np.round(y_b * 127).astype(np.int8).view(np.uint8)
                b8 = np.round(y_h * 127).astype(np.int8).view(np.uint8)

                # construir fila RGB
                fila = np.stack([r8, g8, b8], axis=1)

            else:
                raise ValueError("map_mode debe ser 'ampl', 'fft' o 'fir'")


            # 4.2) Construir imagen segun color_mode
            if self.color_mode == "gris":
                if self.map_mode == "fft":
                    # intercalar mag y phase
                    img = np.empty((N, N), dtype=np.uint8)
                    img[0::2, :] = mag_n[np.newaxis, :]
                    img[1::2, :] = phase_n[np.newaxis, :]
                elif self.map_mode == "fir":
                    # intercalar bandas y repetir verticalmente
                    img = np.empty((N, N), dtype=np.uint8)
                    img[0::3, :] = r8[np.newaxis, :]
                    img[1::3, :] = g8[np.newaxis, :]
                    img[2::3, :] = b8[np.newaxis, :]
                else:
                    gris = fila if fila.ndim == 1 else fila[:,0]
                    img  = np.tile(gris[np.newaxis,:], (N,1))
            else:
                # modo color
                base_rgb = fila if fila.ndim == 2 else np.stack([fila]*3, axis=1)
                colored  = self._colorear_fila(base_rgb, self.map_mode)
                img      = np.tile(colored[np.newaxis,...], (N,1,1))

            # 4.3) Guardar imagen PNG
            path = os.path.join(self.frames_dir, f"frame_{i+1:04d}.png")
            imageio.imwrite(path, img)

        print(f"[Encoder] Frames generados en '{self.frames_dir}/'")


    def encode_video(self, output_name=None):
        """
        Llama a ffmpeg para empaquetar los frames PNG como un MP4 sin perdida.

        - Usa libx264 en modo sin compresion (crf 0)
        - Preset "veryslow" para mejor compresion sin afectar calidad
        - Formato de color: yuv444p
        """
        base = os.path.splitext(os.path.basename(self.audio_path))[0]
        name = output_name or f"{base}_{self.map_mode}_{self.color_mode}.mp4"
        out  = os.path.join(self.export_dir, name)

        # borrar si ya existia
        if os.path.exists(out):
            os.remove(out)

        # construir comando ffmpeg
        cmd = [
            "ffmpeg",
            "-y",                            # sobrescribe sin preguntar
            "-framerate", str(self.fps),    # fps usado en codificacion
            "-i", os.path.join(self.frames_dir, "frame_%04d.png"),  # input por patron
            "-c:v", "libx264",              # codec de video
            "-crf", "0",                    # calidad sin perdida
            "-preset", "veryslow",          # compresion optima
            "-pix_fmt", "yuv444p",          # formato RGB completo
            out
        ]

        # ejecutar ffmpeg
        subprocess.run(cmd, check=True)
        print(f"[Encoder] Video exportado en '{out}'")
        return out

In [2]:
class PixelSoundsDecoder:
    """
    Decodifica un vídeo generado por PixelSoundsEncoder de vuelta a WAV.

    - Primer frame: metadatos (N, hop, fps, color_mode).
    - Siguientes frames: bloques de audio (gris o RGB) codificados.
    - Reconstruye por overlap-add y guarda WAV.
    """
    def __init__(
        self,
        frames_dir,
        output_wav,
        map_mode='ampl',    # 'ampl', 'fft' o 'fir'
        window_type='hann'  # tipo de ventana para overlap-add
    ):
        self.frames_dir  = frames_dir
        self.output_wav  = output_wav
        self.map_mode    = map_mode
        self.window_type = window_type

    def extract_all_frames(self, video_path, prefix="frame_", fmt="png"):
        """
        Extrae todos los frames de un vídeo y los guarda como imágenes PNG numeradas
        en la carpeta self.frames_dir. Limpia el contenido previo si existe.

        Parámetros:
        - video_path: ruta al vídeo del que extraer los frames
        - prefix: prefijo para nombrar los archivos generados
        - fmt: formato de imagen de salida (por defecto 'png')
        """
        print(f"[Decoder] Extrayendo frames de '{video_path}'...")

        # si existe la carpeta, la limpiamos por completo
        if os.path.exists(self.frames_dir):
            shutil.rmtree(self.frames_dir)
        os.makedirs(self.frames_dir, exist_ok=True)

        # abrimos el vídeo
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise IOError(f"No se pudo abrir el vídeo {video_path}")

        # bucle para leer y guardar todos los frames
        idx = 0
        while True:
            ret, frame = cap.read()           # intentamos leer el siguiente frame
            if not ret:
                break                         # fin del vídeo
            fname = f"{prefix}{idx:04d}.{fmt}"  # frame_0000.png, frame_0001.png, ...
            cv2.imwrite(os.path.join(self.frames_dir, fname), frame)
            idx += 1

        cap.release()
        print(f"[Decoder] Total frames extraídos: {idx}")
        return idx


    def extraer_metadatos_cabecera_rows(self, frame):
        """Lee la cabecera y extrae N, hop, fps y si está en color"""
        print("[Decoder] Leyendo metadatos de header...")
        if frame.dtype != np.uint8:
            frame = np.clip(frame * 255, 0, 255).astype(np.uint8)
        if frame.ndim == 3:
            frame = frame[..., 0]  # usar solo canal rojo si es RGB

        # extraer bytes altos y bajos para N y hop
        hiN, loN = int(frame[0,0]), int(frame[1,0])
        hiH, loH = int(frame[2,0]), int(frame[3,0])
        fps      = int(frame[4,0])
        flag     = int(frame[5,0])

        N        = (hiN << 8) | loN
        hop      = (hiH << 8) | loH
        is_color = bool(flag)

        print(f"[Decoder] Header -> N={N}, hop={hop}, fps={fps}, modo={'color' if is_color else 'gris'}")
        return N, hop, fps, is_color

    def decode(self):
        """
        Reconstruye el audio a partir de los PNG ya extraídos en self.frames_dir.
        Asume que frame_0000.png (header) y los frames de datos están presentes.
        """
        # 1) Leer metadatos desde el header
        header = imageio.imread(os.path.join(self.frames_dir, "frame_0000.png"))
        N, hop, fps, self.is_color = self.extraer_metadatos_cabecera_rows(header)
        fs_recon = N * fps

        # 2) Obtener listado de frames ignorando el header
        files   = sorted(f for f in os.listdir(self.frames_dir)
                        if f.startswith("frame_") and f != "frame_0000.png")
        n_blocks = len(files)

        # 3) Inicializar buffers para reconstrucción por superposición (overlap-add)
        length  = N + hop * (n_blocks - 1)
        audio   = np.zeros(length, dtype=np.float32)
        pesos   = np.zeros(length, dtype=np.float32)
        ventana = get_window(self.window_type, N, fftbins=True)

        # 4) Decodificar bloque a bloque
        for i, fname in enumerate(files, start=1):
            path = os.path.join(self.frames_dir, fname)
            raw  = imageio.imread(path)  # imagen uint8

            if self.is_color:
                # === Modo color ===
                row_uint8 = raw[0]  # usamos solo la primera fila

                if self.map_mode == 'ampl':
                    amp_norm = row_uint8[:,0].astype(np.float32) / 255.0
                    block    = amp_norm * 2.0 - 1.0

                elif self.map_mode == 'fft':
                    mag_n    = row_uint8[:,0].astype(np.float32) / 255.0
                    phase_n  = (row_uint8[:,1].astype(np.float32) / 255.0) * 2*np.pi - np.pi
                    X        = mag_n * np.exp(1j * phase_n)
                    X        = np.fft.ifftshift(X)
                    block    = np.real(ifft(X, n=N))

                elif self.map_mode == 'fir':
                    # cada canal representa una banda: low, band, high
                    pix_c = np.ascontiguousarray(row_uint8)
                    y_l   = pix_c[:,0].view(np.int8).astype(np.float32) / 127.0
                    y_b   = pix_c[:,1].view(np.int8).astype(np.float32) / 127.0
                    y_h   = pix_c[:,2].view(np.int8).astype(np.float32) / 127.0
                    block = y_l + y_b + y_h

                else:
                    raise ValueError(f"map_mode desconocido: {self.map_mode}")

            else:
                # === Modo gris ===
                gray = raw[...,0] if raw.ndim == 3 else raw

                if self.map_mode == 'ampl':
                    pix   = gray[0].astype(np.float32)
                    block = (pix / 255.0) * 2.0 - 1.0

                elif self.map_mode == 'fft':
                    mag_n    = gray[0, :].astype(np.float32) / 255.0
                    phase_n  = (gray[1, :].astype(np.float32) / 255.0) * 2*np.pi - np.pi
                    X        = mag_n * np.exp(1j * phase_n)
                    X        = np.fft.ifftshift(X)
                    block    = np.real(ifft(X, n=N))

                elif self.map_mode == 'fir':
                    # filas intercaladas 0::3,1::3,2::3
                    y_l = gray[0::3, :][0].view(np.int8).astype(np.float32) / 127.0
                    y_b = gray[1::3, :][0].view(np.int8).astype(np.float32) / 127.0
                    y_h = gray[2::3, :][0].view(np.int8).astype(np.float32) / 127.0
                    # recombinar bandas
                    block = y_l + y_b + y_h

                else:
                    raise ValueError(f"map_mode desconocido: {self.map_mode}")

            # 4.2) Superposición por ventana
            start = (i-1) * hop
            audio[start:start+N] += block * ventana
            pesos[start:start+N] += ventana

        # 5) Normalizar por ventana y guardar
        audio /= (pesos + 1e-12)
        sonido(audio, fs_recon)  # opcional: reproducir
        os.makedirs(os.path.dirname(self.output_wav) or '.', exist_ok=True)
        scaled = np.int16(np.clip(audio, -1, 1) * 32767)
        wav.write(self.output_wav, fs_recon, scaled)

        return audio, fs_recon


In [7]:
for d in ("fotogramas", "exports"):
    if os.path.exists(d):
        print(f"Eliminando carpeta existente: {d}/")
        shutil.rmtree(d)
    else:
        print(f"No existe: {d}/ — nada que borrar.")

Eliminando carpeta existente: fotogramas/
Eliminando carpeta existente: exports/


## 1. Carga de audio

Cargamos un WAV de ejemplo y mostramos sus primeros segundos.

In [8]:
audio_path = "audios\music.wav"

fs, orig = lee_audio(audio_path)
print(f"Frecuencia de muestreo: {fs} Hz")
Audio(orig, rate=fs)

Frecuencia de muestreo: 44100 Hz


## 2. Definición de parámetros

### Relación entre FPS y FS en PixelSounds

Una parte clave del sistema PixelSounds es la relación entre la frecuencia de muestreo del audio (`fs`) y los fotogramas por segundo (`fps`) del vídeo generado. Esta relación determina cómo se divide la señal de audio en bloques temporales y, por tanto, qué resolución temporal tendrá la visualización.

#### Conversión audio → vídeo

- El audio original tiene una frecuencia de muestreo `fs` (por ejemplo, 44100 Hz).
- El vídeo tendrá `fps` (por ejemplo, 60 fotogramas por segundo).
- Cada frame del vídeo representará un bloque de **N = fs / fps** muestras de audio.

> Por ejemplo, si `fs = 44100` Hz y `fps = 60`, entonces `N = 735`.  
> Esto significa que cada fotograma representa 735 muestras de audio, es decir, 735 / 44100 = 0.0166 segundos ≈ 16.6 ms de sonido.

#### Ventaneo y solapamiento

- Para evitar discontinuidades, se utiliza un solapamiento entre bloques con salto `hop = N // 2` (solapamiento del 50%).
- Cada bloque se multiplica por una ventana (por ejemplo, tipo `hann`) antes de ser codificado para minimizar artefactos.

#### Efecto visual y auditivo

- Un valor alto de `fps` produce más frames por segundo y, por tanto, bloques de audio más pequeños (mayor resolución temporal, más precisión visual).
- Un valor bajo de `fps` produce menos frames por segundo y bloques más grandes (más eficiencia, pero menor detalle en la visualización).

> Es crucial mantener `fs` y `fps` constantes durante todo el proceso para asegurar que la reconstrucción del audio a partir del vídeo sea coherente y sin errores.

Esta correspondencia directa entre parámetros típicos del mundo audiovisual (`fps`) y el dominio de la señal (`fs`) no solo favorece una comprensión intuitiva de la codificación, sino que también resalta cómo el diseño del sistema integra decisiones técnicas con consecuencias visuales y auditivas. Elegir una `fps` adecuada es, por tanto, un equilibrio entre estética visual y fidelidad sonora.

Además, el uso del solapamiento y la ventana aplicada a cada bloque no es meramente un detalle técnico: refleja una comprensión precisa de cómo minimizar discontinuidades entre bloques, técnica comúnmente usada en transformadas de corto tiempo o en codificadores perceptuales.


In [ ]:
fps         = 60
window_type = "hann"  # otras opciones: "hamming", "blackman", "bartlett", "kaiser", "boxcar", "triang", "nuttall", "flattop", "parzen", "bohman"

## 3. Modo Amplitude (`ampl`)


### Codificación

- **Bloque y ventana**  
  Se extrae un fragmento de \( N \) muestras del audio y se aplica una ventana (por ejemplo, Hann) para evitar discontinuidades en los bordes.

- **Normalización a [0–1]**  
  $$
  \text{norm} = \frac{\text{block} - \min(\text{block})}{\max(\text{block}) - \min(\text{block}) + \varepsilon}
  $$  
  Esto asegura que la amplitud mínima del bloque mapee a 0 y la máxima a 1.

- **Cuantización a 8 bits**  
  $$
  \text{fila}_i = \lfloor \text{norm}_i \times 255 \rfloor \quad \text{con} \quad 0 \leq \text{fila}_i \leq 255
  $$

- **Construcción de la imagen**  
  - **Escala de grises**: se replica la misma fila de \( N \) píxeles en cada una de las \( N \) filas del PNG, resultando en una imagen uniforme.
  - **Modo color**:  
    $$
    R = \text{amplitud}, \quad G = 255 - \text{amplitud}, \quad B = 128
    $$

> Aunque este modo es el más sencillo, no se ha descuidado su implementación: se asegura una normalización por bloque que preserva el rango dinámico local, una cuantización sin ambigüedad en 8 bits, y una representación visual coherente y fácilmente reversible. Esta atención al detalle permite que incluso esta versión básica sirva como referencia estable para comparar con las codificaciones más complejas.


In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para ampl (gris y color)
map_mode = "ampl"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    display(Video(video_file, embed=True, width=480, height=360))


---

#### Decodificación


- **Lectura de píxeles**  
  Se carga el PNG y se extrae la fila 0:  
  - En gris → valor único por píxel  
  - En color → canal Rojo

- **Desnormalización a [0–1]**  
  $$
  \text{amp} = \frac{\text{pixel}}{255}
  $$

- **Recuperar bipolaridad [–1 … +1]**  
  $$
  \text{block}_i = \text{amp}_i \times 2 - 1
  $$

- **Overlap-add con ventana**  
  Cada bloque reconstruido se solapa a la mitad (hop = \( N/2 \)) y se suma usando la misma ventana, recomponiendo la señal continua.

- **Normalización final**  
  $$
  \text{audio}[t] \mathrel{{/}{=}} \sum \text{ventana}
  $$

El resultado es un archivo `.wav` cuya forma de onda sigue fielmente la envolvente original, con las únicas pérdidas debidas a la cuantización a 8 bits y al solapamiento de ventanas.

In [ ]:
map_mode = "ampl"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")

## 4. Modo FFT (`fft`)


### Codificación

- **Transformada de Fourier (FFT)**  
  Se aplica una FFT al bloque de audio con longitud \( N \) y se centra el espectro usando un `fftshift`.

  $$
  X = \text{fftshift}\left( \text{FFT}(\text{block}) \right)
  $$

- **Extracción de magnitud y fase**  
  $$
  \text{mag} = |X| \qquad \text{phase} = \angle X
  $$

- **Normalización a enteros sin signo de 8 bits**  
  - Magnitud:

    $$
    \text{mag}_n = \left\lfloor \frac{\text{mag}}{\max(\text{mag}) + \varepsilon} \times 255 \right\rfloor
    $$

  - Fase (rango [\(-\pi\), \(+\pi\)] → [0, 255]):

    $$
    \text{phase}_n = \left\lfloor \frac{\text{phase} + \pi}{2\pi} \times 255 \right\rfloor
    $$

- **Construcción de la imagen**  
  - **Escala de grises**:  
    Se intercalan las filas de magnitud y fase:

    $$
    \text{img}_{2k} = \text{mag}_n \quad , \quad \text{img}_{2k+1} = \text{phase}_n
    $$

  - **Modo color**:  
    Se asigna:

    $$
    R = \text{mag}_n, \quad G = \text{phase}_n, \quad B = 255
    $$

---

> Este modo introduce una complejidad adicional al tener que codificar dos componentes por muestra: magnitud y fase. La elección de normalizar ambas por separado y mapearlas a 8 bits permite mantener la precisión relativa en cada bloque sin necesidad de almacenar escalas globales. El hecho de que la fase esté centrada en cero y remapeada al rango [0, 255] permite una reconstrucción directa desde la imagen, lo cual es poco habitual en representaciones visuales de espectros.

> Además, asignar `R = magnitud`, `G = fase` y `B = 255` en modo color no solo es una decisión práctica sino también visualmente significativa: la magnitud domina la percepción de intensidad, mientras que la fase introduce variaciones suaves en el color, facilitando una inspección visual cualitativa del contenido espectral.


> Accidentalmente, el desarrollo del sistema comenzó sin incluir la fase: solo se codificaba la magnitud del espectro. Esto permitía una reconstrucción parcial del audio, pero el resultado sonoro era artificial y con un timbre metálico característico. Sin saberlo, estábamos recreando el efecto robótico que se explicó en clase al hablar de la importancia de la fase en la percepción del habla y los timbres naturales.

> Esta observación empírica fue clave para darnos cuenta de que la fase debía ser preservada para obtener una reconstrucción fiel. Así nació el modo `fft` completo, que no solo corrige esa limitación sino que también convierte el sistema en un ejemplo práctico y tangible de conceptos vistos en teoría de procesamiento de señal.



In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para FFT (gris y color)
map_mode = "fft"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    # display(Video(video_file, embed=True, width=480, height=360))

### Visualización


Los vídeos generados en modo **FFT** suelen ser muy grandes y a veces el notebook tarda o se “congela” al intentar renderizarlos inline. Por ello **no recomendamos** ejecutar la celda de abajo en el notebook si ves que no responde. 

> **Sugerencia**:  
> - Abre el fichero MP4 directamente en **VLC** o **VSCode**  
> - O puedes reproducirlos en un navegador externo


In [ ]:
base = os.path.splitext(os.path.basename(audio_path))[0]

for map_mode, color in [("fft", "gris"), ("fft", "color")]:
    video_file = os.path.join(
        "exports",
        f"{map_mode}_{color}",
        f"{base}_{map_mode}_{color}.mp4"
    )
    print(f"Vídeo FFT ({map_mode}_{color}): {video_file}")
    # DESCOMENTA la siguiente si quieres intentarlo (no lo recomendamos ) 
    # display(Video(video_file, embed=True, width=480, height=360

### Decodificación

- **Lectura de magnitud y fase**  
  - En modo color se toman los canales R y G.  
  - En modo gris se extraen de las filas 0 (magnitud) y 1 (fase).

- **Desnormalización de magnitud y fase**  
  $$
  \text{mag} = \frac{\text{mag}_n}{255}, \qquad \text{phase} = \frac{\text{phase}_n}{255} \times 2\pi - \pi
  $$

- **Reconstrucción del espectro complejo**  
  $$
  X = \text{mag} \cdot e^{j \cdot \text{phase}}
  $$

- **Transformada inversa y desfase de espectro**  
  $$
  X = \text{ifftshift}(X), \quad \text{block} = \text{Re}\left( \text{IFFT}(X) \right)
  $$

- **Solapamiento y normalización**  
  Se aplica overlap-add con ventana y se normaliza por los pesos acumulados, como en los otros modos.

El modo `fft` permite recuperar tanto el contenido espectral como la fase del bloque, generando reconstrucciones más fieles pero a costa de mayor complejidad.

> A diferencia de representaciones comunes como el espectrograma, que descartan la fase, este sistema la conserva y la reutiliza explícitamente durante la reconstrucción. El resultado es una señal mucho más fiel al original, sin los artefactos metálicos típicos de codificaciones espectrales incompletas.

> Esta simetría entre codificación y decodificación no solo refuerza la robustez del sistema, sino que también proporciona una implementación práctica de lo discutido en clase sobre el papel crucial de la fase en la calidad perceptiva del sonido.


In [ ]:
map_mode = "fft"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")

## 5. Modo FIR (`fir`)


#### Codificación

- **Filtrado en tres bandas**  
  Se aplica un banco de filtros FIR al bloque:

  $$
  y_L = \text{lfilter}(b_{\text{low}}, 1, \text{block}) \\
  y_B = \text{lfilter}(b_{\text{band}}, 1, \text{block}) \\
  y_H = \text{lfilter}(b_{\text{high}}, 1, \text{block})
  $$

- **Clipping y cuantificación a enteros de 8 bits con signo**  
  Se limita cada banda al rango \([-1, +1]\) y se escala a 8 bits con signo:

  $$
  r = \left\lfloor y_L \cdot 127 \right\rfloor, \quad
  g = \left\lfloor y_B \cdot 127 \right\rfloor, \quad
  b = \left\lfloor y_H \cdot 127 \right\rfloor
  $$

  Posteriormente se reinterpretan como `uint8` para almacenarlos en la imagen:

  $$
  r_8 = \text{reinterpretar como uint8}(r), \quad \text{etc.}
  $$

- **Construcción de la imagen**  
  - **Modo color**:  
    Se construye un frame RGB directamente con los canales \((r_8, g_8, b_8)\).

  - **Modo gris**:  
    Se intercalan las tres bandas en las filas de la imagen:

    $$
    \text{img}_{3k} = r_8, \quad \text{img}_{3k+1} = g_8, \quad \text{img}_{3k+2} = b_8
    $$


> Esta codificación propone una lectura creativa del espectro dividiéndolo en tres bandas significativas y mapeándolas directamente a los colores primarios. La baja frecuencia domina el canal rojo, la media el verde, y la alta el azul, lo que permite que cada frame del vídeo adquiera un color característico según la energía distribuida en el tiempo.

> Aunque no conserva la fase ni permite una reconstrucción exacta, este modo tiene una gran fuerza expresiva y es especialmente útil para identificar patrones rítmicos o cambios tímbricos en la señal. El resultado es una representación visual directa e intuitiva, donde incluso una persona sin formación técnica puede "ver" cuándo un sonido es grave, agudo o ruidoso.

> En este sentido, el modo `fir` ejemplifica cómo una idea técnica puede convertirse en una herramienta de análisis perceptual, o incluso en una forma de visualización artística del sonido.


In [ ]:
# 3.1 Generar → Empaquetar → Decodificar para FFT (gris y color)
map_mode = "fir"
for color in ("gris", "color"):
    # 3.1.1 Carpetas de salida
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    os.makedirs(frames_dir, exist_ok=True)
    os.makedirs(export_dir, exist_ok=True)

    # 3.1.2 CODIFICAR
    enc = PixelSoundsEncoder(
        audio_path=audio_path,
        frames_dir=frames_dir,
        export_dir=export_dir,
        fps=fps,
        color_mode=color,
        map_mode=map_mode,
        window_type=window_type
    )
    enc.generate_frames()

    # 3.1.3 EMPAQUETAR VÍDEO
    video_file = enc.encode_video()
    print(f"[Notebook] Vídeo generado en: {video_file}")
    display(Video(video_file, embed=True, width=480, height=360))

---

#### Decodificación

- **Extracción de las bandas**  
  - En **modo color**:

    $$
    y_L = \text{reinterpretar como int8}(R) / 127 \\
    y_B = \text{reinterpretar como int8}(G) / 127 \\
    y_H = \text{reinterpretar como int8}(B) / 127
    $$

  - En **modo gris**:

    $$
    y_L = \text{reinterpretar como int8}(\text{fila }0::3) / 127 \\
    y_B = \text{reinterpretar como int8}(\text{fila }1::3) / 127 \\
    y_H = \text{reinterpretar como int8}(\text{fila }2::3) / 127
    $$

- **Suma de las bandas**  
  Se combinan las tres bandas para reconstruir el bloque:

  $$
  \text{block} = y_L + y_B + y_H
  $$

- **Solapamiento y reconstrucción final**  
  Se hace overlap-add y normalización con la ventana, como en los otros modos.

  Este modo es útil para representar la energía en distintas bandas del espectro, con menor fidelidad que `fft` pero menor tamaño y buena separación frecuencial.

> Este modo mantiene la simetría con la codificación, reinterpretando cada canal como un valor con signo y escalando de vuelta a su rango original. Al sumar las tres bandas recuperadas se obtiene una estimación del bloque original, lo que permite reconstruir la forma general del sonido sin necesidad de conservar fase.

> Aunque no ofrece una fidelidad espectral tan alta como el modo `fft`, su bajo coste computacional y su claridad visual lo convierten en una opción especialmente adecuada para analizar estructuras temporales, acentos o texturas sonoras. Es un ejemplo de cómo una codificación perceptiva —más que matemática— puede ser útil en tareas de análisis, educación musical o visualización artística del sonido.



In [9]:
map_mode = "fir"
base_name = os.path.splitext(os.path.basename(audio_path))[0]

for color in ("gris", "color"):
    # 1) Directorios y rutas
    frames_dir = f"fotogramas/{map_mode}_{color}_fotogramas"
    export_dir = f"exports/{map_mode}_{color}"
    # Nombre del vídeo que acabamos de generar
    video_file = os.path.join(export_dir, f"{base_name}_{map_mode}_{color}.mp4")
    # Fichero WAV de salida
    output_wav = os.path.join(export_dir, f"recon_{base_name}_{map_mode}_{color}.wav")
    
    # 2) Instanciar decoder
    dec = PixelSoundsDecoder(
        frames_dir=frames_dir, # Donde guardar los frames extraidos
        output_wav=output_wav, # Video del que sacar los frames
        map_mode=map_mode,     # Modo
        window_type=window_type # Tipo de ventana
    )
    
    # 3) Extraer frames desde el MP4
    dec.extract_all_frames(video_file)
    
    # 4) Reconstruir el audio y guardarlo
    audio_rec, fs_rec = dec.decode()
    
    # 5) Mostrar y reproducir inline
    print(f"[Notebook] Audio reconstruido ({map_mode}_{color}):")

NameError: name 'window_type' is not defined